# Random Forest Classifier Model
- Trained with VAK qeustions
- 2nd highest accuracy

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.model_selection import GridSearchCV
import joblib

In [2]:
# Load the CSV file containing responses
df = pd.read_csv('Dataset/encoded_new_response_withQues.csv')
df.head()

,Gender,Level of Study,Household Income,Learning Objects [Slide presentation],Learning Objects [Book],Learning Objects [Lecture Note],Learning Objects [Educational game],Learning Objects [Video],Learning Objects [Audio-recorded lecture],Learning Objects [Animated instruction],...,"30. When I'm meeting with an old friend_I say ""it's great to see you!""",Preferred learning mode_Asynchronous Online Learning (On your own time),Preferred learning mode_Face to Face,Preferred learning mode_Synchronous Online Learning (Real Time),Preferred Communication Platform_Call,Preferred Communication Platform_Email,Preferred Communication Platform_Others,Preferred Communication Platform_Telegram,Preferred Communication Platform_University eLearning Chat Room,Preferred Communication Platform_Whatsapp
0,2,1,2,0,0,0,0,0,0,0,...,False,False,True,False,False,False,False,False,False,True
1,2,1,2,0,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,True
2,2,1,2,1,0,1,0,0,0,0,...,True,False,True,False,False,True,False,False,False,False
3,2,1,2,1,0,1,0,0,0,0,...,True,False,True,False,False,False,False,False,True,False
4,2,1,2,1,0,1,0,0,0,0,...,True,False,True,False,False,False,False,False,False,True


### Define target variables

In [3]:
# # Target variable: Learning Objects Preference
target = df[[
    'Learning Objects [Slide presentation]',
    'Learning Objects [Book]',
    'Learning Objects [Lecture Note]',
    'Learning Objects [Educational game]',
    'Learning Objects [Video]',
    'Learning Objects [Audio-recorded lecture]',
    'Learning Objects [Animated instruction]',
    'Learning Objects [Real object model]',
    'Learning Objects [Mind Map]',
    'Learning Objects [Multimedia content]',
    'Learning Objects [Interactive Tool]',
    'Learning Objects [Technology-supported learning include computer-based training systems]',
    'Learning Objects [Intelligent computer-aided instruction systems]'
]]

### Split test and train data

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(target.columns, axis=1), target, test_size=0.25, random_state=42)

### Train model
- use GridSearchCV to find the best parameters which will give the highest accuracy
- save the model using joblib

In [5]:
# Create an empty dictionary to hold the classifiers
best_estimators = {}

# Loop through each learning object
for col in target.columns:
    # Define the parameter grid for hyperparameter tuning
    param_grid = {
#         'n_estimators': [100, 150, 200],  # Vary the number of trees
#         'max_depth': [None, 10, 20, 30],  # Vary the maximum depth of trees
        'n_estimators': [200], 
        'max_depth': [10]
    }
    
    # Instantiate GridSearchCV for RandomForestClassifier
    grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
    
    # Fit the grid search to your data for the current learning object
    grid_search.fit(X_train, y_train[col])
    
    # Get the best parameters and best estimator for the current learning object
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_
    
    # Store the best estimator in the classifiers dictionary
    best_estimators[col] = best_estimator
    
# joblib.dump(best_estimators, "Model/rf_model.joblib")

### With even more parameters
- Take quite a long time to run

In [6]:
# # Create an empty dictionary to hold the classifiers
# best_estimators = {}

# # Loop through each learning object
# for col in target.columns:
#     # Define the parameter grid for hyperparameter tuning
#     param_grid = {
#         'n_estimators': [100, 150, 200, 250],  # Expanded number of trees
#         'max_depth': [None, 10, 20, 30, 50],  # Expanded maximum depth of trees
#         'min_samples_split': [2, 5, 10],  # Minimum samples for node splitting
#         'min_samples_leaf': [1, 2, 4, 8],  # Minimum samples required at leaf nodes
#         'max_features': ['auto', 'sqrt', 'log2']  # Maximum number of features considered for splitting
#         # Add other hyperparameters to tune
#     }
    
#     # Instantiate GridSearchCV for RandomForestClassifier
#     grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')
    
#     # Fit the grid search to your data for the current learning object
#     grid_search.fit(X_train, y_train[col])
    
#     # Get the best parameters and best estimator for the current learning object
#     best_params = grid_search.best_params_
#     best_estimator = grid_search.best_estimator_
    
#     # Store the best estimator in the classifiers dictionary
#     best_estimators[col] = best_estimator
    
#     # Evaluate the model on the validation set
#     y_pred = best_estimator.predict(X_test)
#     accuracy = accuracy_score(y_test[col], y_pred)
#     print(f"Model Accuracy for {col}: {accuracy}")

    
# # Save the best estimators using joblib
# joblib.dump(best_estimators, "Model/rf_model_3.joblib")

In [7]:
print('best_params:', best_params)

best_params: {'max_depth': 10, 'n_estimators': 200}


best_params: {'max_depth': 10, 'n_estimators': 200}

### Check model accuracy
- use classification_report

In [8]:
# rf_model = joblib.load("Model/rf_model.joblib")

In [9]:
# Make predictions on the test set
y_pred = pd.DataFrame({col: classifier.predict(X_test) for col, classifier in best_estimators.items()})

# Classification Report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       492
           1       1.00      0.90      0.95       281
           2       0.95      1.00      0.98       654
           3       1.00      0.90      0.94       343
           4       0.95      0.96      0.95       477
           5       0.98      0.90      0.94       327
           6       0.99      0.91      0.95       365
           7       0.98      0.89      0.94       370
           8       0.99      0.88      0.93       350
           9       0.98      0.93      0.95       428
          10       0.98      0.94      0.96       416
          11       0.99      0.93      0.96       426
          12       0.99      0.92      0.96       392

   micro avg       0.98      0.93      0.95      5321
   macro avg       0.98      0.92      0.95      5321
weighted avg       0.98      0.93      0.95      5321
 samples avg       0.87      0.84      0.84      5321



C:\Users\Janice\anaconda3\envs\dspenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Janice\anaconda3\envs\dspenv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


rf_model:  
micro avg       0.84      0.71      0.77      2101  
macro avg       0.86      0.69      0.76      2101   
weighted avg    0.85      0.71      0.76      2101  
samples avg     0.73      0.61      0.63      2101  

In [10]:
# Initialize a dictionary to store accuracy scores
accuracy_scores = {}

# Loop through each column and calculate accuracy score
for col in y_test.columns:
    accuracy = accuracy_score(y_test[col], y_pred[col])
    accuracy_scores[col] = accuracy
    print(f"Accuracy for {col}: {accuracy}")

# Overall accuracy score
overall_accuracy = accuracy_score(y_test.values.flatten(), y_pred.values.flatten())
print(f"\nOverall Accuracy: {overall_accuracy}")

Accuracy for Learning Objects [Slide presentation]: 0.9547413793103449
Accuracy for Learning Objects [Book]: 0.96875
Accuracy for Learning Objects [Lecture Note]: 0.9644396551724138
Accuracy for Learning Objects [Educational game]: 0.9612068965517241
Accuracy for Learning Objects [Video]: 0.9515086206896551
Accuracy for Learning Objects [Audio-recorded lecture]: 0.9601293103448276
Accuracy for Learning Objects [Animated instruction]: 0.9622844827586207
Accuracy for Learning Objects [Real object model]: 0.9515086206896551
Accuracy for Learning Objects [Mind Map]: 0.9525862068965517
Accuracy for Learning Objects [Multimedia content]: 0.9568965517241379
Accuracy for Learning Objects [Interactive Tool]: 0.9644396551724138
Accuracy for Learning Objects [Technology-supported learning include computer-based training systems]: 0.9644396551724138
Accuracy for Learning Objects [Intelligent computer-aided instruction systems]: 0.9644396551724138

Overall Accuracy: 0.9597977453580901


### Make predictions on a new data

In [14]:
# rf_model = joblib.load("Model/rf_model.joblib")

In [17]:
data = pd.read_csv("Streamlit/merged_withdomVAK.csv")

In [19]:
predictions = pd.DataFrame({col: classifier.predict(data) for col, classifier in best_estimators.items()})
predictions

,Learning Objects [Slide presentation],Learning Objects [Book],Learning Objects [Lecture Note],Learning Objects [Educational game],Learning Objects [Video],Learning Objects [Audio-recorded lecture],Learning Objects [Animated instruction],Learning Objects [Real object model],Learning Objects [Mind Map],Learning Objects [Multimedia content],Learning Objects [Interactive Tool],Learning Objects [Technology-supported learning include computer-based training systems],Learning Objects [Intelligent computer-aided instruction systems]
0,0,0,0,0,0,0,1,1,0,1,1,1,1
1,0,0,0,0,0,0,1,1,1,1,1,1,1
